# 从rl的PPO到RL4HF中的PPO


## 1. 经典 PPO-Clip 公式 (Schulman et al., 2017)

在 OpenAI 提出的 PPO 原始论文 “Proximal Policy Optimization Algorithms” (Schulman et al., 2017) 里，目标函数是：

$$
L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) A_t, \ \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) A_t \right) \right]
$$

其中：

- $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$
- $A_t$ 是优势函数
- $\epsilon$ 是超参数 (如 0.1~0.2)

特点：

- 状态 $s_t$，动作 $a_t$：典型 RL 表达方式

- 没有序列长度归一化

- 没有语言建模的上下文结构，动作是单步离散的

## 关键改动点：

$$
\mathcal{J}_{PPO}(\theta) = \mathbb{E}_{q \sim P(Q), o \sim \pi_{\theta_{\text{old}}}(O|q)} \left[ \frac{1}{|o|} \sum_{t=1}^{|o|} \min \left( \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{\text{old}}}(o_t|q, o_{<t})} A_t, \ \text{clip} \left( \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{\text{old}}}(o_t|q, o_{<t})}, 1 - \epsilon, 1 + \epsilon \right) A_t \right) \right]
$$

- **状态替换**:  
  $s_t \rightarrow (q, o_{<t})$
  - 在语言建模中，状态由输入 prompt 和前缀 token 构成。

- **动作替换**:  
  - $a_t \rightarrow \text{token } o_t$。

- **轨迹归一化**:  
  - 外层有 $\frac{1}{|o|} \sum_t$，表示对一个生成序列内的每个 token 平均，而不是直接累积。

- **采样分布**:  
  - $\mathbb{E}_{q \sim P(Q)}$ 表示从 prompt 分布中抽样（例如 RLHF 数据集中的问题）。

- **概率分布**:  
  - 用的是 语言模型条件概率 $\pi_\theta(o_t|q, o_{<t})$，而不是单步 MDP 动作概率。


论文：

“Training language models to follow instructions with human feedback” (OpenAI, 2022)




### 策略目标(objective)
论文给出的公式为：
$$
\text{objective}(\phi) = \mathbb{E}_{(x,y) \sim D_{\pi_{\phi}^{RL}}} \left[ r_\theta(x, y) - \beta \log \frac{\pi_\phi^{RL}(y \mid x)}{\pi^{SFT}(y \mid x)} \right] + \gamma \mathbb{E}_{x \sim D_{\text{pretrain}}} \left[ \log \pi_\phi^{RL}(x) \right]
$$
说明：  
- 最大化reward，但是使用两种方式进行校准拉回。
  - 第一种是KL的罚项，$- \beta \log \frac{\pi_\phi^{RL}(y \mid x)}{\pi^{SFT}(y \mid x)}$。惩罚离开SFT太远，$\beta$ 是权重越大越保守，越小越最求reward。
  - $\gamma \mathbb{E}_{x \sim D_{\text{pretrain}}} \left[ \log \pi_\phi^{RL}(x) \right]$是为了保持语言模型的基本能力/避免遗忘。保证语言模型的流畅度，防止过分最求reward。
- $D_{\pi_{\phi}^{RL}}$: 按照当前策略采样得到的偏好对(x, y)的分布（先从promt抽出x,然后再从当前策略 $\pi_{\phi}^{RL}$ 生成y）

### 奖励模型的loss

$$
\text{loss}(\theta) \ = \ -\frac{1}{(K/2)} \mathbb{E}_{(x, y_w, y_l) \sim D} \left[ \log \left( \sigma(r_\theta(x, y_w) - r_\theta(x, y_l)) \right) \right]
$$
说明：
- 数据$D$是偏好数据集：对用一种问题prompt-x，有一对回答 $(y_w, y_l)$ winner和loser。
- $\sigma(\cdot)$；$sigmoid()$ 函数
- $-1/(K/2)$ 只是归一化，这里K是总比较数
- 实际操作里，常常把KL散度放到reward里面保持稳定更新

# 总结
- 最初的 PPO = 环境 reward + ratio-clip 稳定更新
- RLHF 版本的 PPO = 奖励模型打分 + 显式 KL 惩罚 + 语言建模正则，本质还是 PPO，但为了文本生成和人类偏好场景做了定制。